In [16]:
# Corpus from https://www.kaggle.com/datasets/chazzer/big-language-detection-dataset?resource=download

In [2]:
import pandas as pd
import numpy as np
import regex as re
import warnings
warnings.simplefilter("ignore")

/var/folders/ct/5yx69gds05q99cq5wg3smhlw0000gq/T/ipykernel_23775/2006435990.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# Preprocess the data set and save in a new file
# This gets rid of punctuation, makes strings into lower case, and drops the 'id' column

"""
data = pd.read_csv("sentences.csv")

data.drop(["id"], axis=1, inplace=True)
data["sentence"] = data.apply(lambda x: re.sub(r"(?=[\p{Common}])[^']|(?<![a-zA-Z])'|'(?![a-zA-Z])", " ", x["sentence"].lower()), axis=1)

data.to_csv("sentences_processed.csv", encoding='utf-8', index=False)
"""

'\ndata = pd.read_csv("sentences.csv")\n\ndata.drop(["id"], axis=1, inplace=True)\ndata["sentence"] = data.apply(lambda x: re.sub(r"(?=[\\p{Common}])[^\']|(?<![a-zA-Z])\'|\'(?![a-zA-Z])", " ", x["sentence"].lower()), axis=1)\n\ndata.to_csv("sentences_processed.csv", encoding=\'utf-8\', index=False)\n'

In [8]:
# Prune the data and get an equal sample size for each language. 
# - The data is filtered to only include 42 of the most spoken languages
# - Right now, that is 28 sentences per language
# 
# This also gets saved as a new file. This could probably be combined with the first step

"""
topLang = ["eng", "cmn", "hin", "spa", "fra", "ara", "ben", "rus", "por", "ind", "urd", "deu", "jpn", "swh", "pnb", "tam", "kor", "vie", "jav", "ita", "tha", "tgl", "pol", "yor", "ukr", "ibo", "npi", "ron", "nld", "zsm", "afr", "grc", "swe", "heb", "lat", "san", "gle", "mri", "chr", "nav", "haw", "smo"]

filtered_data = data[data["lan_code"].isin(topLang)]
filtered_data = filtered_data.groupby("lan_code").sample(28)
filtered_data.to_csv("/content/top_lang_sentences.csv", index=False)

"""

'\ntopLang = ["eng", "cmn", "hin", "spa", "fra", "ara", "ben", "rus", "por", "ind", "urd", "deu", "jpn", "swh", "pnb", "tam", "kor", "vie", "jav", "ita", "tha", "tgl", "pol", "yor", "ukr", "ibo", "npi", "ron", "nld", "zsm", "afr", "grc", "swe", "heb", "lat", "san", "gle", "mri", "chr", "nav", "haw", "smo"]\n\nfiltered_data = data[data["lan_code"].isin(topLang)]\nfiltered_data = filtered_data.groupby("lan_code").sample(28)\nfiltered_data.to_csv("/content/top_lang_sentences.csv", index=False)\n\n'

In [4]:
# Load the preprocessed data
data = pd.read_csv("sentences_processed.csv")

In [21]:
# Encode the labels and retrieve the sentence data
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
labels = le.fit_transform(data["lan_code"])

text_data = data["sentence"]
print(text_data.shape)

(10341812,)


In [22]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

train_text, test_text, train_labels, test_labels = train_test_split(text_data, labels, test_size=0.2)

print(train_text[0])
cv = CountVectorizer()
cv.fit(train_text)

train_text = cv.transform(train_text)
test_text = cv.transform(test_text)

我們試試看 


In [23]:
print(train_text.shape)

(8273449, 2004014)


In [11]:
from sklearn.naive_bayes import MultinomialNB
from dask_ml.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix

model = GaussianNB()
model.fit(train_text, train_labels)

# predictions = model.predict(test_text)

# ac = accuracy_score(test_labels, predictions)
# cm = confusion_matrix(test_labels, predictions)

# print("accuracy: " + ac)
# print("matrix: " + cm)

AttributeError: 'csr_matrix' object has no attribute 'var'